## Geneious analysis for individual samples from raw Geneious output, "Annotation.csv" 

 ### Required packages
 - No specific package required 
 
 ### Inputs
 - Geneious SNP analysis of _k13_, _crt_, _mdr1_, _dhfr_, _dhps_, and _cytb_
 - Documentation on Geneious analysis can be found: Readme.md
 - Geneious outputs were modified to GuineaAnalysis_Individual.csv from "Annotation.csv"
 
 
 ### Data structure 
 - [Long-form](https://seaborn.pydata.org/tutorial/data_structure.html#long-form-vs-wide-form-data) 
     - Each variable is a column 

         - "Sample" = *AMD ID*, including associated meta-data for each sample
             - AMD ID and bit code key is found under MS Teams > Domestic > Files > Sample Naming > Sample_naming_key.pptx  

             - Key: **Year Country State/Site DayofTreatment Treatment SampleID Genus SampleType GeneMarker-8bitcode SampleSeqCount**

                 - Example:
                     - Individual sequenced sample ID: 17GNDo00F0001PfF1290 = 2017 Guinea Dorota Day0 AS+AQ 0001 P.falciparum FilterBloodSpot k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47 

                     - Pooled sequenced sample ID: 17GNDoxxx001P10F1290 = 2017 Guinea Dorota **xx x** 001 **Pooled SamplesInPool** P.falciparum FilterBloodSpot k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47 

                         - NOTE: If information is not availble (na) **x** is used. For pooled samples, DayofTreatment and Treatment is na since its a pool of multiple samples with that info. 
                         - NOTE: For pooled samples, **Genus** is replaced with **Pooled** and **SampleType** with **SamplesInPool** to indicated this as a pooled sequenced sample and sample count in each pool. 
         <p>&nbsp;</p>
         - "Year" = the year the study was conducted 
         - "Site" = the state or province 
         - "Day_of_treatment" = describes the day of treatment provided to the patient 
         - "Gene" = drug resistant gene(s) 
         - "G_annotation" = full SNP annotation in the following format: WildTypeAA-CodonPosition-MutantAA 
         - "Coverage" = the number of reads covering the SNP 
         - "VAF" = variant allele frequency calculated by AA divided by total reads in loci 
         - "SNP" = single nucleotide polymorphism in WildTypeAA or MutantAA annotation format 
         - "Type" = describes if it is a wild type or mutant SNP 

     - Each observation is a row for each sample ID (patient ID) 
 
 #### TODO
 
 #### Activity Name
 - [ ] Write doc.string at the beginning of the code
 - [ ] Write detailed description with comment for line by line
 - [ ] Make the code more simple and accurate
 - [ ] Follow zen of python
    
 #### Completed Activity ✓
 - [x] Created marked down at the beginning of the file for description

In [3]:
#######This document is for individual samples report for the EPI team#######

"""
This is individual_geneious_EP jupyter notebook file. This file is for processing raw Geneious output file 
"Annotation.csv" file into GuineaAnalysis_Individual.csv file which contains information filtered such as 
Sample,Pooled,Year,SITE,TreatmentDay,GENE,G_annotation,COVERAGE,VAF,VF,SNP,TYPE. Sample means AMD_ID which
has naming schema including other informations within the name. Pooled means whether the sample is an individual
sample or pooled sample. This jupyter notebook first assigns variants. Then, it checks for silent mutations. 
At last it adds full wildtypes. The differnce from Lab individual jupyter notebook is that this file uses one
additional dictionary to filter out SNPs that are not reportable.
"""

######First I create a dictionary of reportable SNPs so that I could later use it to filter out unreportable SNP samples####
dict_reportables_filter={}  ##Create a dictionary for filtering reportable SNPs
#####The voinew3.csv file contains a table of reportable SNPs########
with open ("voinew3.csv", "r") as v1:  ##voinew3.csv file contains information reportable SNPs 
    ####I loop through lines in the voi file ###
    for lines in v1:  
        ###I would like to skip the header####
        if "Ref" not in lines:
            ######I add Gene name and SNP####
            dict_reportables_filter[lines.split(",")[2]+lines.split(",")[3]+lines.split(",")[4].strip("\n")]="exist"

###Open raw output table file "Annotations.csv" from Geneious#####
import itertools
###### Assign letters to the amino acid names #########
#####This is a dictionary assigning three letters to amino acid letter #####
AAdic = {  ###Amino Acid dictionary which assigns an amino acid letters to three letters of amino acid names####
"Ala": "A",
"Arg": "R",
"Asn": "N",
"Asp": "D",
"Asx": "B",
"Cys": "C",
"Glu": "E",
"Gln": "Q",
"Glx": "Z",
"Gly": "G",
"His": "H",
"Ile": "I",
"Leu": "L",
"Lys": "K",
"Met": "M",
"Phe": "F",
"Pro": "P",
"Ser": "S",
"Thr": "T",
"Trp": "W",
"Tyr": "Y",
"Val": "V",
"STOP": "X",
}
#######Convert combination of bases to a codon############
######This is coverting three base combination into a coddon####
#####It would create dicionary of three bases conversion to codons######
Current_Codon = ""
Codon_Combinations = ["".join(i) for i in itertools.product(["T", "G", "A", "C"], repeat=3)]  ####Create combination of
### 4 letters into three letters
Codon_Dic = {}  ### Create a dictionary for codon
for x in Codon_Combinations:  ###Loop over combinations
    Current_Codon = ""   ###At beginning of loop current codon is assigned as empty string
    for y in x:  ###For letter in combination
        Current_Codon += y  ###Add each letter from the combination to form the current codon
    if Current_Codon == "TTT" or Current_Codon == "TTC":
        Codon_Dic[Current_Codon] = "Phe"
    if Current_Codon == "TTA" or Current_Codon == "TTG":
        Codon_Dic[Current_Codon] = "Leu"
    if (
        Current_Codon == "CTT"
        or Current_Codon == "CTC"
        or Current_Codon == "CTA"
        or Current_Codon == "CTG"
    ):
        Codon_Dic[Current_Codon] = "Leu"
    if Current_Codon == "ATT" or Current_Codon == "ATC" or Current_Codon == "ATA":
        Codon_Dic[Current_Codon] = "Ile"
    if Current_Codon == "ATG":
        Codon_Dic[Current_Codon] = "Met"
    if (
        Current_Codon == "GTT"
        or Current_Codon == "GTC"
        or Current_Codon == "GTA"
        or Current_Codon == "GTG"
    ):
        Codon_Dic[Current_Codon] = "Val"
    if (
        Current_Codon == "TCT"
        or Current_Codon == "TCC"
        or Current_Codon == "TCA"
        or Current_Codon == "TCG"
    ):
        Codon_Dic[Current_Codon] = "Ser"
    if (
        Current_Codon == "CCT"
        or Current_Codon == "CCC"
        or Current_Codon == "CCA"
        or Current_Codon == "CCG"
    ):
        Codon_Dic[Current_Codon] = "Pro"
    if (
        Current_Codon == "ACT"
        or Current_Codon == "ACC"
        or Current_Codon == "ACA"
        or Current_Codon == "ACG"
    ):
        Codon_Dic[Current_Codon] = "Thr"
    if (
        Current_Codon == "GCT"
        or Current_Codon == "GCC"
        or Current_Codon == "GCA"
        or Current_Codon == "GCG"
    ):
        Codon_Dic[Current_Codon] = "Ala"
    if Current_Codon == "TAT" or Current_Codon == "TAC":
        Codon_Dic[Current_Codon] = "Tyr"
    if Current_Codon == "CAT" or Current_Codon == "CAC":
        Codon_Dic[Current_Codon] = "His"
    if Current_Codon == "CAA" or Current_Codon == "CAG":
        Codon_Dic[Current_Codon] = "Gln"
    if Current_Codon == "AAT" or Current_Codon == "AAC":
        Codon_Dic[Current_Codon] = "Asn"
    if Current_Codon == "AAA" or Current_Codon == "AAG":
        Codon_Dic[Current_Codon] = "Lys"
    if Current_Codon == "GAT" or Current_Codon == "GAC":
        Codon_Dic[Current_Codon] = "Asp"
    if Current_Codon == "GAA" or Current_Codon == "GAG":
        Codon_Dic[Current_Codon] = "Glu"
    if Current_Codon == "TGT" or Current_Codon == "TGC":
        Codon_Dic[Current_Codon] = "Cys"
    if Current_Codon == "TGG":
        Codon_Dic[Current_Codon] = "Trp"
    if (
        Current_Codon == "AGA"
        or Current_Codon == "AGG"
        or Current_Codon == "CGT"
        or Current_Codon == "CGC"
        or Current_Codon == "CGA"
        or Current_Codon == "CGG"
    ):
        Codon_Dic[Current_Codon] = "Arg"
    if Current_Codon == "AGT" or Current_Codon == "AGC":
        Codon_Dic[Current_Codon] = "Ser"
    if (
        Current_Codon == "GGT"
        or Current_Codon == "GGC"
        or Current_Codon == "GGA"
        or Current_Codon == "GGG"
    ):
        Codon_Dic[Current_Codon] = "Gly"
    if Current_Codon == "TGA" or Current_Codon == "TAA" or Current_Codon == "TAG":
        Codon_Dic[Current_Codon] = "STOP"
        
filter_exist_dict={}##Create a dictionary to filter out duplicate information. For example when there is information about variant for a sample, meaning variant exsits,
##Then we do not need to consider having full wildtypes
#####Open Geneious raw output file which is Annotation.csv####
####This is the ulfiltered file######
with open ("Annotations.csv", "r") as r1:  ###Import raw geneious output for data cleaning and analysis purpose
    ####We will write filterd information on GuineaAnalysis_individual_EP######
    with open("GuineaAnalysis_Individual_EP.csv" , "w") as w1:
        ####We will write the header first with given information#####
        w1.write("Sample,Pooled,Year,SITE,TreatmentDay,GENE,G_annotation,COVERAGE,VAF,VF,SNP,TYPE\n")  ##Write columns for the information for the cleaned data
        ####We will create a dictionary with area code in it so that####
        ####We can infer name of region from the sample ID#####
        dict1={}
        dict1["Ha"]="Hamdalaye"
        dict1["Do"]="Dorota"
        dict1["Ma"]="Maferinyah"
        dict1["La"]="Lay-Sarè"
        dict1["LS"]="Lay-Sarè"
        dict2={}
        #####We will loop through each line in the raw Geneious output file######
        for lines in r1:   ##run a for loop for each lines in Annotation.csv to analyze line by line
            ######Check if there is Polymorphism in the line from the table#######
            ######If there is polymorphism then that means there is variant######
            ######According to Geneious as geneious report variants as polymorphisms####
            ######We will loop through Geneious raw output for variants first####
            if "Polymorphism" in lines and "Variants" in lines:  ##Check if two words, polymorphism and variants, are in the lines. These words indicate
                ##There are variants of interest in the line, Thus, we would like to extract the information the given line
                ####sample name is first column right before the _ string#####
                ####We will filer out unnecessary information from sample name#####
                samplename=lines.split(",")[0].split("_")[0]  ##We get sample name which is first part before string "_"
                ####Sample gene is the second coulmmn in the line####
                ###We will assign genes to the given variants of interest####
                samplegene=lines.split(",")[1]  ##The gene is found in the second column split by ","
                ###We want to correct name for DHPS####
                if "DHPS" in samplegene:  ##Fix name for DHPS
                    samplegene="DHPS"
                ###We want to correct name of MT CYTB####
                if "mitochondrial" in samplegene:  ##Fix name for cytob
                    samplegene="CYTB"
                ###Sample AA change is column 17 which has the amino acid changes####
                sampleAAchange=lines.split(",")[16]   ##We identify AA change which is 17th column
                ####If there is Amino acid change reported combine it with the c variable which is codon position####
                ####to create a SNP change report #####
                if sampleAAchange!="":  ##If we find AA change at 17th column we want to use that information to build SNP
                    ####Based on spacing you could figurout aminoacid letters for before changing after changing#####
                    a=sampleAAchange.split(" ")[0]  ##Wildtype amino acid
                    b=sampleAAchange.split(" ")[2]  ##mutation amino acid
                    ####C is the aminoacid position where change has happened######
                    c=lines.split(",")[20]  ##amino acid  position
                    AAchangereport=(a+c+b)  ## If you add a+c+b then SNP is obtained
                    ###Next We will assign variant allele frequency to a column####
                    sampleVAF=(lines.split("M05039")[0]).split(",")[-4]  ##Four coulms before M05039 finds the variant allele frequency
                    ###Next We will assign variant frequency to the next column####
                    sampleVF=(lines.split("M05039")[0]).split(",")[-2]   ##Two columns before M05039 finds the variant frequency
                    ####We want to assign coverage information to the next column####
                    sampleCoverage=lines.split(",")[28]   ##The coverage of sample is found on 29th column
                    ###We want to make sure these are Guinea sample which has GN in its name####
                    if "GN" in samplename:  ##Check if this is guinea sample and not a control
                        ####Use a dictionary to prevent overlaps######
                        ####We want to use condition that there shouldn't be letter p in that position#####
                        ####To identify individual samples also we don't want to process duplicates such as ####
                        ####duplicate wildtypes or variants once we processed them because there can be only either variant or wildtype####
                        ####Also for EPI report the SNP has to be in the previously created dictionary#####
                        ####We want to make the dictionary without the last letter which is changed amino acid because ####
                        ####We want to prevent situation such as same amino acid change and same wildtype but different mutation change #####
                        if samplename[9]!="p" and (samplename,AAchangereport[0:-1]) not in filter_exist_dict and AAchangereport in dict_reportables_filter:  ##Check if this sample is not pooled
                            ##In addition check if it is a reportable SNPs
                            ####We want to assign to dictionary3 for preventing duplicates in the future####
                            filter_exist_dict[samplename,AAchangereport[0:-1]]="exist"   ##Assign a dictionary so that we won't repeat assigning to create duplicates
                            #####If the 1A is in that postion of sample ID we assign treatment day 1#####
                            #####We assign information based on the header we wrote previously#####
                            if samplename[6:8]=="1A":  ##Check treatment day that if it is 1A in given position it means that it was treated on the first day
                                w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+"01"+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport[1::]+","+"mutation"+"\n")
                            ####Otherwise we assign the treatment day based on name####
                            else:  ##Else we assign treatment day based on given position of the sample name
                                w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+samplename[6:8]+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport[1::]+","+"mutation"+"\n")
                #####For silent mutation when there is no amino acid change####
                #####We check if there is mutation change or not####
                ####If there is no amino acid change we considered it silent mutation####
                if sampleAAchange=="" and lines.split(",")[26]!="":  ##This part is for assigning silent mutation
                    ####For silent mutation we have to use three base changes instead of aminoacid letter change###
                    ####Due to how Geneious reports it#####
                    ####We use the previously created codon dictionary and amino acid dictionary to get the amino acid letter for the three bases changes####
                    a=AAdic[Codon_Dic[lines.split(",")[26][0:3]]]  ##Assign a amino acid using codon dictionary and amino acid dictionary createed previously
                    ##This is the wildtype part of the SNP
                    b=AAdic[Codon_Dic[lines.split(",")[26][-3::]]]  ##Assign a mutation part of the SNP using codon dictionary and amino acid dictionary 
                    ####C is the location of amino acid change####
                    c=lines.split(",")[20]  ##Assign a amino acid location for the given SNP
                    AAchangereport=(a+c+b)
                    ####Again we get the sample variant allele frequency and variant frequency depending on the location in the geneious raw output table#### 
                    sampleVAF=(lines.split("M05039")[0]).split(",")[-4]  ##Four coulms before M05039 finds the variant allele frequency
                    sampleVF=(lines.split("M05039")[0]).split(",")[-2]  ##Two columns before M05039 finds the variant frequency
                    ####We also get the average coverage depednign on which column it is###
                    ####I need to further investigate using average coverage vs maximum coverage####
                    sampleCoverage=lines.split(",")[28]   ##The coverage of sample is found on 29th column
                        ####Use a dictionary to prevent overlaps######
                    if samplename[9]!="p" and (samplename,AAchangereport[0:-1]) not in filter_exist_dict and AAchangereport in dict_reportables_filter:  ##Check if this sample is not pooled
                        ##In addition check if it is a reportable SNPs
                        ####We want to assign to dictionary3 for preventing duplicates in the future####
                        filter_exist_dict[samplename,AAchangereport[0:-1]]="exist"  ##Prevent duplicates information
                        #####If the 1A is in that postion of sample ID we assign treatment day 1#####
                        #####We assign information based on the header we wrote previously#####
                        if samplename[6:8]=="1A":  ##Check for the treatment day of 1
                            w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+samplename[6:8]+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport+","+"silent"+"\n")
                        ####Otherwise we assign the treatment day based on name####
                        else:  ##Check for the treatment day later than 1
                            w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+samplename[6:8]+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport+","+"silent"+"\n")

####We open the annotation file again for identifying pure wildtpye this time####
with open ("Annotations.csv", "r") as r1:   ##Import geneious raw output again
    with open("GuineaAnalysis_Individual_EP.csv" , "a") as w1:  ##Add full widtype information to the table
        ####We go through each line in the annotation file####
        for lines in r1:
            ####This time conditional would be only for coverage####
            ####If there is coverage and it is not 0 that means####
            ####It is wildtype#####
            if "Coverage:" in lines:  ##If coverage exists in the line we consider it wildtype if that is not variant
                ####sample name is first column right before the _ string#####
                ####We will filer out unnecessary information from sample name#####
                samplename=lines.split(",")[0].split("_")[0]
                ####Sample gene is the second coulmmn in the line####
                ###We will assign genes to the given wildtypes of interest####
                samplegene=lines.split(",")[1]
                ###We want to correct name for DHPS####
                if "DHPS" in samplegene:  ##correct dhps name
                    samplegene="DHPS"
                ###We want to correct name of MT CYTB####
                if "mitochondrial" in samplegene:  ##correct my cytob name
                    samplegene="CYTB"
                ####For lines containing Coverage instead of polymorphism###
                ###The aminoacid change is reported at different column####
                ###It is reported at the last column as trackerSNP####
                AAchangereport=lines[0:-12].split(",")[-1]  ##check SNP
                ###Since this is wildtype sampleVAF would be 0%####
                ###Sample variant frequency is also 0#####
                sampleVAF="0%"  ##sample VAF is 0% because this is full wildtype
                sampleVF="0"  ##sample variant frequency is 0 because this is full wildtype
                ####For lines containing Coverage instead of polymorphism###
                ###The sample coverage is reported at different column####
                sampleCoverage=lines.split(",")[17]  ##assign sample coverage
                ####Again we are only interested in Guinea samples####
                if "GN" in samplename:  ##check if this is guinea sample
                    ####Since this is individual report for EPI team we are gonna find individuals that are not p####
                    ####at a given position of the naming schema###
                    if samplename[9]!="p" and (samplename,AAchangereport[0:-1]) not in filter_exist_dict and AAchangereport in dict_reportables_filter:  ##Check if this sample is not pooled
                        ##In addition check if it is a reportable SNPs
                        ###We want to setup a dictionary so that we wouldn't create duplicate report from the variant report made previously#####
                        filter_exist_dict[samplename,AAchangereport[0:-1]]="exist"  ##check if variant sample is already assigned before assigning full wildtype
                        ####If 1A is in the naming schema we treat it as treatment day of 1######
                        if samplename[6:8]=="1A":   ##check treatment day
                            w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+"01"+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport[0:-1]+","+"wildtype"+"\n")
                        ####Otherwise we just assign the given treatment day in the naming schema#####
                        else:
                            w1.write(samplename+","+"Individual"+","+samplename[0:2]+","+dict1[samplename[4:6]]+","+samplename[6:8]+","+samplegene+","+AAchangereport+","+sampleCoverage+","+sampleVAF+","+sampleVF+","+AAchangereport[0:-1]+","+"wildtype"+"\n")